In [112]:
import pandas as pd
import numpy as np
import chardet
import matplotlib.pyplot as plt
import seaborn as sns
import streamlit as st
import pyarrow as pa
import pyarrow.parquet as pq
import time
import csv
import io
from concurrent.futures import ThreadPoolExecutor, as_completed
from functools import partial

In [ ]:
# folder_path = input("Enter folder path : ")
# os.chdir(folder_path)
# os.getcwd()

# Old Function 

In [35]:
def clean_dataset(filename):
    """
    filename : Is the name of the csv that has the data from the sensor
    Example : A1_right hip_102373_2024-07-31 10-52-18 
    NOTE: There is no need to add .csv in front of the filename 
    """
    df = pd.read_csv(filename + '.csv', skiprows=99)
#     df.drop(df.columns[[5,7,8,9,10,11,12,15]], axis=1, inplace = True)  
    df.drop(df.columns[[5,7,8,9,10,11,12]], axis=1, inplace = True)    
    df.rename(columns={'Unnamed: 0': 'Timing', 'Overall class ': 'Overall class','360 angle ':'360 angle','Supine-recline class ':'Supine-recline class'}, inplace=True)
    df['Timing'] = pd.to_datetime(df['Timing'], format='%Y-%m-%d %H:%M:%S:%f')
    df['Milliseconds'] = df['Timing'].dt.microsecond // 1000
    df['Time Diff'] = df['Milliseconds'].diff().fillna(10.0)
    df.loc[df['Time Diff'] < 0, 'Time Diff'] = 10.0
    return df

# Reading Data 

In [34]:
df.columns

Index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9',
       'Unnamed: 10', 'Unnamed: 11', 'Unnamed: 12'],
      dtype='object')

In [65]:
# df = processed_file_analysis('E1_right hip_102373_2024-08-27 14-57-00_processed_sheet.csv')

df2 = pd.read_csv('E1_right hip_102373_2024-08-27 14-57-00_processed_sheet.csv')

C:\Users\saira\AppData\Local\Temp\ipykernel_14740\2629178982.py:3: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df2 = pd.read_csv('E1_right hip_102373_2024-08-27 14-57-00_processed_sheet.csv')


In [114]:
def processed_file_analysis(filename):
    df = pd.read_csv(filename + '.csv')
    df.rename(columns={'A': 'Timing'}, inplace=True)
    df['Timing'] = pd.to_datetime(df['Timing'], format='%Y-%m-%d %H:%M:%S:%f')
    df['Milliseconds'] = df['Timing'].dt.microsecond // 1000
    df['Time Diff'] = df['Milliseconds'].diff().fillna(10.0)
    df.loc[df['Time Diff'] < 0, 'Time Diff'] = 10.0
    
    return df

In [111]:
df.head()

,Timing,B,C,D,360 angle,Up/down angle,Body Rotation,Prone-sit class,Supine-recline class,Overall class,Milliseconds,Time Diff
0,2024-08-22 10:16:32.000,-0.0159,-0.517,-0.8473,178.924943,-31.385951,supine-recline,NaN,right side,right side,0,10.0
1,2024-08-22 10:16:32.010,-0.0360,-0.513,-0.8315,177.520913,-31.648845,supine-recline,NaN,right side,right side,10,10.0
2,2024-08-22 10:16:32.020,-0.0400,-0.521,-0.8354,177.258700,-31.920416,supine-recline,NaN,right side,right side,20,10.0
3,2024-08-22 10:16:32.030,-0.0199,-0.525,-0.8196,178.609124,-32.634184,supine-recline,NaN,right side,right side,30,10.0
4,2024-08-22 10:16:32.040,-0.0279,-0.513,-0.8038,178.012054,-32.531130,supine-recline,NaN,right side,right side,40,10.0


In [74]:
df2.columns[7:9]

Index(['Prone-sit class', 'Supine-recline class'], dtype='object')

In [101]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8878118 entries, 0 to 8878117
Data columns (total 10 columns):
 #   Column                Dtype  
---  ------                -----  
 0   A                     object 
 1   B                     float64
 2   C                     float64
 3   D                     float64
 4   360 angle             float64
 5   Up/down angle         float64
 6   Body Rotation         object 
 7   Prone-sit class       object 
 8   Supine-recline class  object 
 9   Overall class         object 
dtypes: float64(5), object(5)
memory usage: 677.3+ MB


In [99]:
df2[~df2['Prone-sit class'].isna()]

,A,B,C,D,360 angle,Up/down angle,Body Rotation,Prone-sit class,Supine-recline class,Overall class
87,2024-08-22 10:16:32:870,0.0403,-0.3770,-0.8909,182.590018,-22.915496,prone-sit,prone supported,NaN,prone supported
88,2024-08-22 10:16:32:880,0.0644,-0.4250,-0.9067,184.062713,-25.058670,prone-sit,upright,NaN,upright
89,2024-08-22 10:16:32:890,0.1527,-0.6649,-0.4632,198.245487,-53.739023,prone-sit,upright,NaN,upright
90,2024-08-22 10:16:32:900,0.2170,-0.5250,-0.8711,193.988257,-30.319684,prone-sit,upright,NaN,upright
91,2024-08-22 10:16:32:910,0.1768,-0.7089,-0.6968,194.237280,-44.599516,prone-sit,upright,NaN,upright
...,...,...,...,...,...,...,...,...,...,...
8878108,2024-08-23 10:59:13:900,0.0604,-0.9969,0.0199,288.235511,-86.349960,prone-sit,sitting,NaN,sitting
8878109,2024-08-23 10:59:13:910,0.0644,-0.9649,0.0357,299.001699,-85.636116,prone-sit,sitting,NaN,sitting
8878110,2024-08-23 10:59:13:920,0.0604,-0.9689,0.0397,303.316309,-85.733694,prone-sit,sitting,NaN,sitting
8878111,2024-08-23 10:59:13:930,0.0283,-1.0009,0.0397,324.516968,-87.211301,prone-sit,sitting,NaN,sitting


In [98]:
df2[df2['Prone-sit class'].isna()]

,A,B,C,D,360 angle,Up/down angle,Body Rotation,Prone-sit class,Supine-recline class,Overall class
0,2024-08-22 10:16:32:000,-0.0159,-0.5170,-0.8473,178.924943,-31.385951,supine-recline,NaN,right side,right side
1,2024-08-22 10:16:32:010,-0.0360,-0.5130,-0.8315,177.520913,-31.648845,supine-recline,NaN,right side,right side
2,2024-08-22 10:16:32:020,-0.0400,-0.5210,-0.8354,177.258700,-31.920416,supine-recline,NaN,right side,right side
3,2024-08-22 10:16:32:030,-0.0199,-0.5250,-0.8196,178.609124,-32.634184,supine-recline,NaN,right side,right side
4,2024-08-22 10:16:32:040,-0.0279,-0.5130,-0.8038,178.012054,-32.531130,supine-recline,NaN,right side,right side
...,...,...,...,...,...,...,...,...,...,...
8878113,2024-08-23 10:59:13:950,-0.0119,-1.0089,0.0397,16.686003,-87.647639,supine-recline,NaN,reclined,reclined
8878114,2024-08-23 10:59:13:960,-0.0159,-1.0169,0.0476,18.471052,-87.174670,supine-recline,NaN,reclined,reclined
8878115,2024-08-23 10:59:13:970,-0.0199,-1.0249,0.0397,26.622751,-87.518958,supine-recline,NaN,reclined,reclined
8878116,2024-08-23 10:59:13:980,-0.0239,-1.0249,0.0357,33.801034,-87.599689,supine-recline,NaN,reclined,reclined


In [96]:
df2.groupby('Prone-sit class')['Prone-sit class'].count()

Prone-sit class
prone               735838
prone supported    2328475
sitting             648918
upright            1226978
Name: Prone-sit class, dtype: int64

In [95]:
gp_cnt

,index,Prone-sit class
0,prone supported,2328475
1,upright,1226978
2,prone,735838
3,sitting,648918


In [88]:
df2['Prone-sit class'].isna().sum()

3937909

In [92]:
gp_cnt = df2['Prone-sit class'].value_counts().reset_index()

In [76]:
df2['Prone-sit class'].isna().sum()

3937909

In [78]:
df2['Prone-sit class'].count() - df2['Prone-sit class'].isna().sum()

1002300

In [15]:
process_dataset('E1_right hip_102373_2024-08-27 14-57-00.csv')

Processing complete. Results saved as E1_right hip_102373_2024-08-27 14-57-00_processed_sheet.csv in the current folder


# Identifying BottleNeck

In [117]:
import cProfile
import pstats

def main():
    process_dataset('E1_right hip_102373_2024-08-27 14-57-00.csv')

cProfile.run('main()', 'stats')

p = pstats.Stats('stats')
p.sort_stats('cumulative').print_stats(10)

Processing complete. Results saved as E1_right hip_102373_2024-08-27 14-57-00_processed_sheet.csv in the current folder
Total rows read: 8878119
Rows successfully processed: 0
Invalid rows skipped: 8878119
Thu Aug 29 15:26:32 2024    stats

         6117024187 function calls in 6222.778 seconds

   Ordered by: cumulative time
   List reduced from 115 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.001    0.001 6222.777 6222.777 {built-in method builtins.exec}
        1    0.000    0.000 6222.776 6222.776 <string>:1(<module>)
        1    0.189    0.189 6222.776 6222.776 C:\Users\saira\AppData\Local\Temp\ipykernel_14740\2863022495.py:4(main)
        1  113.335  113.335 6222.587 6222.587 C:\Users\saira\AppData\Local\Temp\ipykernel_14740\4150426416.py:47(process_dataset)
  8878119   47.819    0.000 6108.177    0.001 C:\Users\saira\AppData\Local\Temp\ipykernel_14740\4150426416.py:3(process_row)
  8878119   67.106    0.00

In [121]:
import memory_profiler
from memory_profiler import profile

@profile
def process_dataset(file):

    output_file = file.rsplit('.', 1)[0] + '_processed_sheet.csv'
    
    with open(file, 'rb') as infile, open(output_file, 'w', newline='') as outfile:
        content = infile.read()
        
        text_content = content.decode('utf-8', errors='replace')
        
        reader = csv.reader(io.StringIO(text_content))
        writer = csv.writer(outfile)
        
        for _ in range(100):
            next(reader, None)
        
        writer.writerow(['A','B','C','D','360 angle','Up/down angle',
                         'Body Rotation','Prone-sit class','Supine-recline class','Overall class'])
        
        process_row_partial = partial(process_row)
        
        for row in reader:
            if not row:  
                continue
            
            processed_row = process_row_partial(row)
            if processed_row:
                writer.writerow(processed_row)

# Debugging

While writing back data remove the following columns :
Timing,Milliseconds,Time Diff

**Note** : Handle missed data type

In [113]:
def process_row(row, S5=0):
    if len(row) < 3:
        return None  
    
    try:
        B, C, D = map(float, row[:3])
    except ValueError:
        return None  # Skip rows where B, C, D can't be converted to float
    
    angle_360 = np.sign(B) * np.arccos(-D / np.sqrt(B**2 + D**2)) * 180 / np.pi + 180
    angle_updown = np.arcsin(C / np.sqrt(B**2 + C**2 + D**2)) * 180 / np.pi
    body_rotation = "supine-recline" if S5 < angle_360 < (S5 + 180) else "prone-sit"
    
    if body_rotation == "prone-sit":
        if angle_updown > 0:
            prone_sit_class = "prone"
        elif angle_updown > -23:
            prone_sit_class = "prone supported"
        elif angle_updown > -63:
            prone_sit_class = "upright"
        else:
            prone_sit_class = "sitting"
        supine_recline_class = ""
    else:
        if angle_updown > 15:
            supine_recline_class = "upsidedown"
        elif angle_updown < -36:
            supine_recline_class = "reclined"
        elif angle_360 < (S5 + 69):
            supine_recline_class = "left side"
        elif angle_360 > (S5 + 101):
            supine_recline_class = "right side"
        else:
            supine_recline_class = "supine"
        prone_sit_class = ""
    
    overall_class = prone_sit_class + supine_recline_class
    
    return [B, C, D, angle_360, angle_updown, body_rotation, prone_sit_class, supine_recline_class, overall_class]

def process_dataset(file):
    output_file = file.rsplit('.', 1)[0] + '_processed_sheet.csv'
    
    with open(file, 'rb') as infile, open(output_file, 'w', newline='') as outfile:
        # Read the file in binary mode
        content = infile.read()
        
        # Try to decode with utf-8, replacing errors
        text_content = content.decode('utf-8', errors='replace')
        
        # Create a CSV reader from the decoded content
        reader = csv.reader(io.StringIO(text_content))
        writer = csv.writer(outfile)
        
        # Skip header rows
        for _ in range(100):
            next(reader, None)
        
        # Write header
        writer.writerow(['B','C','D','360 angle','Up/down angle',
                         'Body Rotation','Prone-sit class','Supine-recline class','Overall class'])
        
        process_row_partial = partial(process_row)
        
        for row in reader:
            processed_row = process_row_partial(row)
            if processed_row:
                writer.writerow(processed_row)
    
    print(f"Processing complete. Results saved as {output_file} in the current folder")


def dataset_description(df):
    
    class_counts = df['Overall class'].fillna('NaN').groupby(df['Overall class'].fillna('Missing Rows')).count().reset_index(name='Class Count')
    class_counts['Duration in seconds'] = class_counts['Class Count'] / 100
    class_counts = class_counts[['Overall class', 'Duration in seconds']]
    
    total_duration = class_counts['Duration in seconds'].sum()
    # print(f"Duration of Video: {total_duration}")
    
    # statistics = df.describe()
    return class_counts, total_duration

def create_plot(df):
    class_counts = df['Overall class'].fillna('NaN').groupby(df['Overall class'].fillna('Missing Rows')).count().reset_index(name='Class Count')
    class_counts['Duration in seconds'] = class_counts['Class Count'] / 100
    
    fig, ax = plt.subplots(figsize=(8, 6))  # Create figure and axes objects
    bars = ax.bar(class_counts['Overall class'], class_counts['Duration in seconds'])
    
    for bar in bars:
        yval = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2, yval + 1, round(yval, 2), ha='center', va='bottom')
    
    ax.set_xticklabels(class_counts['Overall class'], rotation=45)
    ax.set_yticks([])
    ax.set_xlabel('Category')
    ax.set_ylabel('Duration in Seconds')
    sns.despine(ax=ax, bottom=True, left=True)

    plt.show()
    
    # st.pyplot(fig)  # Provide the figure to st.pyplot

def plot_bins(df, class_name):
    same_class_mask = df['Overall class'] == df['Overall class'].shift(1)
    df['Increment'] = np.where(same_class_mask, 10, 0)
    df['Rolling Sum'] = df['Increment'].groupby((~same_class_mask).cumsum()).cumsum() / 1000
    df.drop(columns=['Increment'], inplace=True)
    
    d = df[df['Overall class'] == class_name].copy()
    
    # If there are no rows for the given class
    if d.empty:
        return("No values for class '{class_name}' exist.")
#         st.warning(f"No values for class '{class_name}' exist.")
#         return
    
    max_val = d['Rolling Sum'].max()
    
    fixed_bins = [0.1, 0.2, 0.3, 0.4, 0.5, 1]
    variable_bins = np.linspace(1.5, max(max_val, 1.5), num=5)
    bins = np.unique(np.sort(np.concatenate((fixed_bins, variable_bins))))
    
    d['duration_bin'] = pd.cut(d['Rolling Sum'], bins, include_lowest=True)
    
    cnt_bin = d.groupby(['Overall class', 'duration_bin']).size().reset_index(name='bin_count')
    
    # Check if all bin counts are zero
    if cnt_bin['bin_count'].sum() == 0:
        st.warning(f"No values for class '{class_name}' exist.")
        return
    
    cnt_bin['duration_bin'] = cnt_bin['duration_bin'].astype(str)
    
    fig, ax = plt.subplots(figsize=(10, 6))
    bars = ax.bar(x='duration_bin', height='bin_count', data=cnt_bin)
    
    for bar in bars:
        yval = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2, yval + 1, round(yval, 2), ha='center', va='bottom')
    
    ax.set_title(f"Buckets for: {class_name}")
    ax.set_xticklabels(cnt_bin['duration_bin'], rotation=45, ha='right')
    ax.set_yticks([])
    ax.set_xlabel('Duration (seconds)')
    ax.set_ylabel('Count')
    sns.despine(ax=ax, bottom=True, left=True)
    plt.tight_layout()

    plt.show()
    # st.pyplot(fig)
    
def overall_class_stats(df, overall_class):
    class_indices = df[df['Overall class'] == overall_class].index
    cnt_arr = []
    cnt = max_cnt = 1
    start = end = class_indices[0]
    
    for i in range(len(class_indices) - 1):
        if class_indices[i + 1] == class_indices[i] + 1:
            cnt += 1
            max_cnt = max(cnt, max_cnt)
        else:
            end = class_indices[i]
            cnt_arr.append((cnt, start, end))
            start = class_indices[i + 1]
            cnt = 1
    
    cnt_arr.append((cnt, start, end))  # To account for the last sequence
    max_sequence = max(cnt_arr, key=lambda x: x[0])
    
    return max_sequence

In [53]:
def process_row(row, S5=0):
    if len(row) < 3:
        return None, f"Insufficient data: {row}"
    
    try:
        B, C, D = map(float, row[:3])
    except ValueError:
        return None, f"Non-numeric data: {row[:3]}"
    
    # Check if B, C, D are all zero
    if B == 0 and C == 0 and D == 0:
        return None, "All zero values"
    
    angle_360 = np.sign(B) * np.arccos(-D / np.sqrt(B**2 + D**2)) * 180 / np.pi + 180
    angle_updown = np.arcsin(C / np.sqrt(B**2 + C**2 + D**2)) * 180 / np.pi
    body_rotation = "supine-recline" if S5 < angle_360 < (S5 + 180) else "prone-sit"
    
    if body_rotation == "prone-sit":
        if angle_updown > 0:
            prone_sit_class = "prone"
        elif angle_updown > -23:
            prone_sit_class = "prone supported"
        elif angle_updown > -63:
            prone_sit_class = "upright"
        else:
            prone_sit_class = "sitting"
        supine_recline_class = ""
    else:
        if angle_updown > 15:
            supine_recline_class = "upsidedown"
        elif angle_updown < -36:
            supine_recline_class = "reclined"
        elif angle_360 < (S5 + 69):
            supine_recline_class = "left side"
        elif angle_360 > (S5 + 101):
            supine_recline_class = "right side"
        else:
            supine_recline_class = "supine"
        prone_sit_class = ""
    
    overall_class = prone_sit_class + supine_recline_class
    
    return [B, C, D, angle_360, angle_updown, body_rotation, prone_sit_class, supine_recline_class, overall_class], None

def process_dataset(file):
  
    output_file = file.rsplit('.', 1)[0] + '_processed_sheet.csv'
    debug_file = file.rsplit('.', 1)[0] + '_debug_info.txt'
    
    total_rows = 0
    processed_rows = 0
    empty_rows = 0
    invalid_rows = 0
    
    with open(file, 'rb') as infile, \
         open(output_file, 'w', newline='', encoding='utf-8') as outfile, \
         open(debug_file, 'w', encoding='utf-8') as debugfile:
        
        # Read the file in binary mode
        content = infile.read()
        
        # Try to decode with utf-8, replacing errors
        text_content = content.decode('utf-8', errors='replace')
        
        # Create a CSV reader from the decoded content
        reader = csv.reader(io.StringIO(text_content))
        writer = csv.writer(outfile)
        
        # Skip header rows
        for _ in range(100):
            next(reader, None)
        
        # Write header
        writer.writerow(['B','C','D','360 angle','Up/down angle',
                         'Body Rotation','Prone-sit class','Supine-recline class','Overall class'])
        
        process_row_partial = partial(process_row)
        
        for row in reader:
            total_rows += 1
            if not row:  # Check for completely empty rows
                empty_rows += 1
                debugfile.write(f"Row {total_rows}: Empty row\n")
                continue
            
            processed_row, error = process_row_partial(row)
            if processed_row:
                writer.writerow(processed_row)
                processed_rows += 1
            else:
                invalid_rows += 1
                debugfile.write(f"Row {total_rows}: {error}\n")
            
            if total_rows <= 10 or total_rows % 1000 == 0:
                debugfile.write(f"Row {total_rows}: {row[:10]}\n")  # Write first 10 elements of each row
    
    print(f"Processing complete. Results saved as {output_file} in the current folder")
    print(f"Debug information saved as {debug_file} in the current folder")
    print(f"Total rows read: {total_rows}")
    print(f"Rows successfully processed: {processed_rows}")
    print(f"Empty rows skipped: {empty_rows}")
    print(f"Invalid rows skipped: {invalid_rows}")

# Usage
process_dataset('E1_right hip_102373_2024-08-27 14-57-00.csv')

Processing complete. Results saved as E1_right hip_102373_2024-08-27 14-57-00_processed_sheet.csv in the current folder
Debug information saved as E1_right hip_102373_2024-08-27 14-57-00_debug_info.txt in the current folder
Total rows read: 8878119
Rows successfully processed: 0
Empty rows skipped: 0
Invalid rows skipped: 8878119


In [27]:
pd.read_csv('E1_right hip_102373_2024-08-27 14-57-00_processed_sheet.csv')

,B,C,D,360 angle,Up/down angle,Body Rotation,Prone-sit class,Supine-recline class,Overall class


Missing Rows after processing the file 

In [116]:
csv.field_size_limit(int(1e9))  # Set to a very large number (1 billion)

def process_row(row, S5=0):
    try:
        # Read the first column and convert the next three elements to float
        A = pd.to_datetime(row[0])
        B, C, D = map(float, row[1:4])
    except ValueError as e:
#         print(f"Non-numeric data: {row[:4]}")  # Print the first four elements for context
        return None

    # Skip rows where B, C, and D are all zero
    if B == 0 and C == 0 and D == 0:
        return None
    
    angle_360 = np.sign(B) * np.arccos(-D / np.sqrt(B**2 + D**2)) * 180 / np.pi + 180
    angle_updown = np.arcsin(C / np.sqrt(B**2 + C**2 + D**2)) * 180 / np.pi
    body_rotation = "supine-recline" if S5 < angle_360 < (S5 + 180) else "prone-sit"
    
    if body_rotation == "prone-sit":
        if angle_updown > 0:
            prone_sit_class = "prone"
        elif angle_updown > -23:
            prone_sit_class = "prone supported"
        elif angle_updown > -63:
            prone_sit_class = "upright"
        else:
            prone_sit_class = "sitting"
        supine_recline_class = ""
    else:
        if angle_updown > 15:
            supine_recline_class = "upsidedown"
        elif angle_updown < -36:
            supine_recline_class = "reclined"
        elif angle_360 < (S5 + 69):
            supine_recline_class = "left side"
        elif angle_360 > (S5 + 101):
            supine_recline_class = "right side"
        else:
            supine_recline_class = "supine"
        prone_sit_class = ""
    
    overall_class = prone_sit_class + supine_recline_class
    
    return [A, B, C, D, angle_360, angle_updown, body_rotation, prone_sit_class, supine_recline_class, overall_class]

def process_dataset(file):
    """
    Process a large dataset from a CSV file.
    
    :param file: Name of the CSV file (with .csv extension)
    """
    output_file = file.rsplit('.', 1)[0] + '_processed_sheet.csv'
    
    total_rows = 0
    processed_rows = 0
    empty_rows = 0
    invalid_rows = 0
    
    with open(file, 'rb') as infile, open(output_file, 'w', newline='') as outfile:
        # Read the file in binary mode
        content = infile.read()
        
        # Try to decode with utf-8, replacing errors
        text_content = content.decode('utf-8', errors='replace')
        
        # Create a CSV reader from the decoded content
        reader = csv.reader(io.StringIO(text_content))
        writer = csv.writer(outfile)
        
        # Skip header rows
        for _ in range(100):
            next(reader, None)
        
        # Write header
        writer.writerow(['A','B','C','D','360 angle','Up/down angle',
                         'Body Rotation','Prone-sit class','Supine-recline class','Overall class'])
        
        process_row_partial = partial(process_row)
        
        for row in reader:
            total_rows += 1
            if not row:  # Check for completely empty rows
                empty_rows += 1
                continue
            
            processed_row = process_row_partial(row)
            if processed_row:
                writer.writerow(processed_row)
                processed_rows += 1
            else:
                invalid_rows += 1
    
    print(f"Processing complete. Results saved as {output_file} in the current folder")
    print(f"Total rows read: {total_rows}")
    print(f"Rows successfully processed: {processed_rows}")
#     print(f"Empty rows skipped: {empty_rows}")
    print(f"Invalid rows skipped: {invalid_rows}")

In [63]:
csv.field_size_limit(int(1e9))  # Set to a very large number (1 billion)

def process_row(row, S5=0):
    try:
        # Read the first column and convert the next three elements to float
        A = row[0]
        B, C, D = map(float, row[1:4])
    except ValueError as e:
#         print(f"Non-numeric data: {row[:4]}")  # Print the first four elements for context
        return None

    # Skip rows where B, C, and D are all zero
    if B == 0 and C == 0 and D == 0:
        return None
    
    angle_360 = np.sign(B) * np.arccos(-D / np.sqrt(B**2 + D**2)) * 180 / np.pi + 180
    angle_updown = np.arcsin(C / np.sqrt(B**2 + C**2 + D**2)) * 180 / np.pi
    body_rotation = "supine-recline" if S5 < angle_360 < (S5 + 180) else "prone-sit"
    
    if body_rotation == "prone-sit":
        if angle_updown > 0:
            prone_sit_class = "prone"
        elif angle_updown > -23:
            prone_sit_class = "prone supported"
        elif angle_updown > -63:
            prone_sit_class = "upright"
        else:
            prone_sit_class = "sitting"
        supine_recline_class = ""
    else:
        if angle_updown > 15:
            supine_recline_class = "upsidedown"
        elif angle_updown < -36:
            supine_recline_class = "reclined"
        elif angle_360 < (S5 + 69):
            supine_recline_class = "left side"
        elif angle_360 > (S5 + 101):
            supine_recline_class = "right side"
        else:
            supine_recline_class = "supine"
        prone_sit_class = ""
    
    overall_class = prone_sit_class + supine_recline_class
    
    return [A, B, C, D, angle_360, angle_updown, body_rotation, prone_sit_class, supine_recline_class, overall_class]

def process_dataset(file):
    """
    Process a large dataset from a CSV file.
    
    :param file: Name of the CSV file (with .csv extension)
    """
    output_file = file.rsplit('.', 1)[0] + '_processed_sheet.csv'
    
    total_rows = 0
    processed_rows = 0
    empty_rows = 0
    invalid_rows = 0
    
    with open(file, 'rb') as infile, open(output_file, 'w', newline='') as outfile:
        # Read the file in binary mode
        content = infile.read()
        
        # Try to decode with utf-8, replacing errors
        text_content = content.decode('utf-8', errors='replace')
        
        # Create a CSV reader from the decoded content
        reader = csv.reader(io.StringIO(text_content))
        writer = csv.writer(outfile)
        
        # Skip header rows
        for _ in range(100):
            next(reader, None)
        
        # Write header
        writer.writerow(['A','B','C','D','360 angle','Up/down angle',
                         'Body Rotation','Prone-sit class','Supine-recline class','Overall class'])
        
        process_row_partial = partial(process_row)
        
        for row in reader:
            total_rows += 1
            if not row:  # Check for completely empty rows
                empty_rows += 1
                continue
            
            processed_row = process_row_partial(row)
            if processed_row:
                writer.writerow(processed_row)
                processed_rows += 1
            else:
                invalid_rows += 1
    
    print(f"Processing complete. Results saved as {output_file} in the current folder")
    print(f"Total rows read: {total_rows}")
    print(f"Rows successfully processed: {processed_rows}")
#     print(f"Empty rows skipped: {empty_rows}")
    print(f"Invalid rows skipped: {invalid_rows}")

# Usage
process_dataset('E1_right hip_102373_2024-08-27 14-57-00.csv')

Processing complete. Results saved as E1_right hip_102373_2024-08-27 14-57-00_processed_sheet.csv in the current folder
Total rows read: 8878119
Rows successfully processed: 8878118
Invalid rows skipped: 1


In [ ]:
# Vectorized some of the operations to reduce total time taken to process the file.

csv.field_size_limit(int(1e9))  # Set to a very large number 

def process_row(df_row, S5=0):
    A = df_row[0] 
    try:
        B, C, D = map(float, df_row[1:4])
    except ValueError:
        return [A, None, None, None, None, None, None, None, None, None]
    
    if B == 0 and C == 0 and D == 0:
        return [A, B, C, D, None, None, None, None, None, None]
    
    angle_360 = np.sign(B) * np.arccos(-D / np.sqrt(B**2 + D**2)) * 180 / np.pi + 180
    angle_updown = np.arcsin(C / np.sqrt(B**2 + C**2 + D**2)) * 180 / np.pi
    body_rotation = "supine-recline" if S5 < angle_360 < (S5 + 180) else "prone-sit"
    
    if body_rotation == "prone-sit":
        if angle_updown > 0:
            prone_sit_class = "prone"
        elif angle_updown > -23:
            prone_sit_class = "prone supported"
        elif angle_updown > -63:
            prone_sit_class = "upright"
        else:
            prone_sit_class = "sitting"
        supine_recline_class = ""
    else:
        if angle_updown > 15:
            supine_recline_class = "upsidedown"
        elif angle_updown < -36:
            supine_recline_class = "reclined"
        elif angle_360 < (S5 + 69):
            supine_recline_class = "left side"
        elif angle_360 > (S5 + 101):
            supine_recline_class = "right side"
        else:
            supine_recline_class = "supine"
        prone_sit_class = ""
    
    overall_class = prone_sit_class + supine_recline_class
    
    return [A, B, C, D, angle_360, angle_updown, body_rotation, prone_sit_class, supine_recline_class, overall_class]

def process_dataset(file):
    """
    Process a large dataset from a CSV file using pandas for efficient I/O.
    
    :param file: Name of the CSV file (with .csv extension)
    """
    output_file = file.rsplit('.', 1)[0] + '_processed_sheet.csv'
    
    df = pd.read_csv(file, skiprows=100, header=None)
    processed_data = df.apply(lambda row: process_row(row), axis=1)
    
    processed_df = pd.DataFrame(processed_data.tolist(), columns=['A (String)', 'B', 'C', 'D', '360 angle', 'Up/down angle', 'Body Rotation', 'Prone-sit class', 'Supine-recline class', 'Overall class'])
    processed_df.to_csv(output_file, index=False)
    
    print(f"Processing complete. Results saved as {output_file} in the current folder")
    print(f"Total rows read: {len(df)}")
    print(f"Rows successfully processed: {len(processed_df)}")

In [110]:
df = processed_file_analysis('E1_right hip_102373_2024-08-27 14-57-00_processed_sheet')

C:\Users\saira\AppData\Local\Temp\ipykernel_14740\545540891.py:5: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename + '.csv')


In [115]:
import cProfile
import pstats

def main():
    process_dataset('E1_right hip_102373_2024-08-27 14-57-00.csv')

cProfile.run('main()', 'stats')

p = pstats.Stats('stats')
p.sort_stats('cumulative').print_stats(10)

Processing complete. Results saved as E1_right hip_102373_2024-08-27 14-57-00_processed_sheet.csv in the current folder
Thu Aug 29 13:39:43 2024    stats

         17756383 function calls in 47.105 seconds

   Ordered by: cumulative time
   List reduced from 36 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.005    0.005   47.105   47.105 {built-in method builtins.exec}
        1    0.000    0.000   47.100   47.100 <string>:1(<module>)
        1    0.427    0.427   47.099   47.099 C:\Users\saira\AppData\Local\Temp\ipykernel_14740\2863022495.py:4(main)
        1   24.255   24.255   46.673   46.673 C:\Users\saira\AppData\Local\Temp\ipykernel_14740\3082505536.py:41(process_dataset)
  8878119   19.970    0.000   20.845    0.000 C:\Users\saira\AppData\Local\Temp\ipykernel_14740\3082505536.py:1(process_row)
  8878121    0.875    0.000    0.875    0.000 {built-in method builtins.len}
        1    0.723    0.723    0.723    